In [34]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets
from sklearn.preprocessing import scale
import numpy as np
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold

In [3]:
train_data = datasets.load_boston()

In [6]:
type(train_data)

sklearn.datasets.base.Bunch

In [11]:
features = train_data.feature_names
print features

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [23]:
# obtain features and target
features = train_data.data
target = train_data.target


In [26]:
# scaling features
features = scale(features)


In [31]:

param_set = np.linspace(1,10,num=200)
print param_set[:10]

[ 1.          1.04522613  1.09045226  1.13567839  1.18090452  1.22613065
  1.27135678  1.31658291  1.36180905  1.40703518]


In [112]:
def param_iterator(param_list, model=KNeighborsRegressor):
    '''takes the list of parameters to iterate over and create regressors w/ those parameters'''
    
    model_list = {}
    for elem in param_list:
        regressor = model(n_neighbors=5, weights='distance', p=elem)
        model_list.update({elem: regressor})
    
    return model_list



In [114]:
# build the list of KNR with different p's
knr_list = param_iterator(param_set)


In [157]:
def get_crossValScore(model_list):
    '''takes the model w/ different parameters and evaluates the cross validation score'''
    model_list_keys = model_list.keys()
    model_list_values = model_list.values()
    
    score_list = []
    
    score_dict = {}
    
    for model in model_list_values:
        score = cross_val_score(model, features, y=target, scoring='mean_squared_error')
        score_list.append(score)
        
    
    score_dict.update(elem: score)
        
    
    
   
    
    return score_list
        
#validator = cross_val_score(kn_regressor, features, scoring='mean_squared_error')
#model_accuracy = KFold(n_folds=5, random_state=42, shuffle=True)

SyntaxError: invalid syntax (<ipython-input-157-3d3f2ef531e3>, line 10)

In [116]:
score_list = get_crossValScore(knr_list)


TypeError: estimator should a be an estimator implementing 'fit' method, 1.0 was passed

In [105]:
score_list_mean = []
for elem in score_list:
    score_list_mean.append(elem.mean())

# get the maximum score mean from the list and associated p
max(score_list_mean)

-33.411037912906806

In [125]:
a = {1: 'a', 2: 'b'}

In [147]:
a= [1,2,3]
b= {}
c = 'a'
a = b.fromkeys(a,c)
print a


{1: 'a', 2: 'a', 3: 'a'}
